# Razor Diffusion Plot + Hex Facet Index + Shear Capacity (v1)

This notebook:

- Loads a JSONL run (default: `../runs/demo.jsonl`)
- Normalizes fields required by the RDM evaluator
- Computes RDM / RDM* for the run
- Evaluates an adversarial cheating baseline
- Plots instantaneous semantic drift per unit cost
- Computes Hex Facet Index metrics (facet IDs, leap rate, Lattice Stability Score)
- Visualizes facet occupancy as a hexbin density map
- Computes Shear Capacity (SC) as a metric-only overhead diagnostic

**Note:** If `../runs/demo.jsonl` is missing, the notebook will explain what to do instead of crashing.


In [ ]:
# Ensure imports work when running from /notebooks
import os, sys

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Repo root:", repo_root)


In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

from razor_metrics.rdm import compute_rdm
from baselines.cheating_agent import run_cheating_agent

# Hex facet index utilities (requires razor_metrics/facets.py)
from razor_metrics.facets import (
    embedding_to_facet,
    HexFacetConfig,
    facet_distance,
)

# Shear Capacity (SC) utilities (requires razor_metrics/shear.py)
from razor_metrics.shear import compute_shear_capacity, ShearConfig


In [ ]:
# Load a normal benchmark run
# Default path: ../runs/demo.jsonl
# If missing, we provide a clear message rather than crashing.

demo_path = os.path.join(repo_root, "runs", "demo.jsonl")

steps = None
if not os.path.exists(demo_path):
    print("\n⚠️  Demo run file not found:")
    print("  ", demo_path)
    print("\nWhat to do:")
    print("  1) Create the folder:  runs/")
    print("  2) Add a JSONL file:  runs/demo.jsonl")
    print("\nEach line should be a JSON object with at least:")
    print("  - embedding: list[float]")
    print("  - cost: number  (or tokens, which will be mapped to cost)")
    print("Optional (enables SC):")
    print("  - boundary: float in [0,1]")
    print("\nStopping here so you can add a run file and re-run the notebook.")
else:
    with open(demo_path, "r", encoding="utf-8") as f:
        steps = [json.loads(line) for line in f if line.strip()]

    print("Loaded steps:", len(steps))
    print("First keys:", sorted(list(steps[0].keys())) if steps else "<empty>")


In [ ]:
# Normalize fields so compute_rdm() does not crash on minimal logs.
#
# Expected by compute_rdm (typical):
# - embedding: list[float]
# - cost: numeric (tokens or proxy)
# - memory_similarity: float
# - violations: int
# - progress: float
#
# If your demo.jsonl uses different names (e.g., tokens), we map them here.

def normalize_steps(steps):
    for s in steps:
        # cost: prefer explicit cost, else tokens, else 1
        if "cost" not in s:
            if "tokens" in s:
                s["cost"] = float(s.get("tokens", 1))
            else:
                s["cost"] = 1.0

        # governance fields (safe defaults)
        s.setdefault("memory_similarity", 0.0)
        s.setdefault("violations", 0)
        s.setdefault("progress", 0.0)

        # Basic embedding sanity check (fail early with readable message)
        if "embedding" not in s:
            raise KeyError(
                "Each step must include an 'embedding' field (list[float]). "
                "Your JSONL is missing it."
            )
        if not isinstance(s["embedding"], list) or len(s["embedding"]) == 0:
            raise ValueError("'embedding' must be a non-empty list[float].")

# Only run normalization if steps were loaded
if steps is not None:
    normalize_steps(steps)
    print("Normalized. Example step:")
    print({k: steps[0][k] for k in ["cost", "memory_similarity", "violations", "progress"]})


In [ ]:
# Compute RDM metrics for the normal run
if steps is not None:
    metrics = compute_rdm(steps)
    metrics


In [ ]:
# --- Shear Capacity (SC) (v1 diagnostic) ---
# SC estimates the fraction of total compute cost that is non-core overhead.
# Requires per-step "boundary" values to be meaningful.
# If boundary is missing, SC will be NaN (undefined) by design.

if steps is not None:
    sc = compute_shear_capacity(steps, ShearConfig(core_boundary_threshold=0.80))
    print("Shear Metrics (SC)")
    print("  Total cost:", sc["total_cost"])
    print("  Core cost:", sc["core_cost"])
    print("  Shear Capacity (SC):", sc["shear_capacity"])


In [ ]:
# Evaluate adversarial cheating baseline
if steps is not None:
    cheat_run = run_cheating_agent()

    # Ensure cheating run has the same normalized fields (future-proof)
    normalize_steps(cheat_run)

    cheat_metrics = compute_rdm(cheat_run)
    cheat_metrics


In [ ]:
# Plot instantaneous semantic drift per unit cost for the normal run
# compute_rdm typically annotates steps[i]['delta_t'] for i>=1.

if steps is not None:
    deltas = [s.get("delta_t", 0.0) for s in steps[1:]]
    costs  = [float(s.get("cost", 1.0)) for s in steps[1:]]

    inst = np.array(deltas, dtype=float) / np.maximum(1.0, np.array(costs, dtype=float))

    plt.figure()
    plt.plot(inst)
    plt.title("Instantaneous Semantic Drift per Unit Cost")
    plt.xlabel("Step")
    plt.ylabel("Δ / cost")
    plt.tight_layout()
    plt.show()


In [ ]:
# --- Hex Facet Index metrics (v1) ---
# Hex facets discretize latent motion into a stable lattice.
# We measure:
# - unique facet occupancy (semantic tourism)
# - facet step distance (continuity)
# - leap rate (d>1)
# - lattice stability score (LSS = 1 - leap_rate)

if steps is not None:
    cfg = HexFacetConfig(cell_size=0.25, seed=1337)
    facet_ids = [embedding_to_facet(s["embedding"], cfg) for s in steps]

    dists = [facet_distance(facet_ids[i-1], facet_ids[i]) for i in range(1, len(facet_ids))]

    unique_facets = len(set(facet_ids))
    mean_facet_step = float(np.mean(dists)) if dists else 0.0
    leap_rate = (sum(1 for d in dists if d > 1) / max(1, len(dists))) if dists else 0.0
    lss = 1.0 - leap_rate

    print("Hex Facet Metrics")
    print("  Unique facets visited:", unique_facets)
    print("  Mean facet step distance:", mean_facet_step)
    print("  Leap rate (d>1):", leap_rate)
    print("  Lattice Stability Score (LSS):", lss)


In [ ]:
# --- Hex facet occupancy visualization (density) ---
# Project facet IDs into 2D for plotting: (q, r)

if steps is not None:
    qs = np.array([q for (q, r) in facet_ids], dtype=float)
    rs = np.array([r for (q, r) in facet_ids], dtype=float)

    plt.figure()
    plt.hexbin(qs, rs, gridsize=35)
    plt.title("Hex Facet Occupancy — (q, r) density")
    plt.xlabel("facet q")
    plt.ylabel("facet r")
    plt.tight_layout()
    plt.show()


In [ ]:
# Quick comparison summary (human-readable)
def summarize(label, m):
    keys = ["RDM", "RDM_star", "A", "D_T", "C_T"]
    out = {k: m.get(k) for k in keys}
    print(label)
    for k, v in out.items():
        print(f"  {k}: {v}")
    print()

if steps is not None:
    summarize("Normal run", metrics)
    summarize("Cheating baseline", cheat_metrics)
    print("(Hex metrics above apply to the normal run.)")
